In [1]:
# installing 1.0.3 because this version of pandas supports write to s3
!pip install pandas==1.0.3

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# This path will be active after the launch of the hackathon / day-day-up-unsw / s3://tf-trachack-notebooks/day-day-up-unsw/jupyter/jovyan/
teamname = 'day-day-up-unsw' #'trachack-a-groups-admin-py-tracfone'
data_folder='s3://tf-trachack-data/212/'
# change root_folder to your team's root folder
# s3://tf-trachack-notebooks/<this should be replaced by team name as provided in EMAIL>/jupyter/jovyan/
root_folder='s3://tf-trachack-notebooks/'+teamname+'/jupyter/jovyan/'

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from datetime import datetime
from pandas import to_datetime

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [4]:
# upgrades=pd.read_csv(data_folder+"data/dev/upgrades.csv")
# upgrades_eval=pd.read_csv(data_folder+"data/eval/upgrades.csv")

In [5]:
phone_info = pd.read_csv(root_folder+"code/data/dev/phone_info.csv")
phone_info_eval = pd.read_csv(root_folder+"code/data/eval/phone_info_eval.csv")
train_set = pd.read_csv(root_folder+"code/data/dev/upgrade_redemptions_deactivations_reactivations_suspensions.csv")
eval_set = pd.read_csv(root_folder+"code/data/eval/upgrade_redemptions_deactivations_reactivations_suspensions_eval.csv")
customer_info = pd.read_csv(root_folder+"code/data/dev/customer_info_new.csv")
customer_info_eval = pd.read_csv(root_folder+"code/data/eval/customer_info_new_eval.csv")

In [6]:
# train_set = pd.merge(upgrades, train_set, on='line_id')
# eval_set = pd.merge(eval_set, phone_info, on='line_id')
train_set = pd.merge(train_set, phone_info, on='line_id')
train_set = pd.merge(train_set, customer_info, on='line_id')
eval_set = pd.merge(eval_set, phone_info_eval, on='line_id')
eval_set = pd.merge(eval_set, customer_info_eval, on='line_id')

In [7]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55868 entries, 0 to 55867
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   line_id                    55868 non-null  object 
 1   upgrade                    55868 non-null  float64
 2   redemption_year            55868 non-null  float64
 3   redemption_month           55868 non-null  float64
 4   redemption_day             55868 non-null  float64
 5   redemptions_0              55868 non-null  float64
 6   redemptions_1              55868 non-null  float64
 7   redemptions_2              55868 non-null  float64
 8   redemptions_3              55868 non-null  float64
 9   redemptions_4              55868 non-null  float64
 10  redemptions_5              55868 non-null  float64
 11  ave_gross_revenue          55868 non-null  float64
 12  deactivation_year          55868 non-null  float64
 13  deactivation_month         55868 non-null  flo

In [8]:
eval_set.head(5)

,line_id,redemption_year,redemption_month,redemption_day,redemptions_0,redemptions_1,redemptions_2,redemptions_3,redemptions_4,redemptions_5,...,customer_redemption_day,plan_0,plan_1,plan_2,plan_3,plan_4,plan_5,carrier_0,carrier_1,carrier_2
0,584f0d75-5897-4e56-b475-0b3adf1aeb07,1.421779,1.473173,1.560174,0.0,0.0,0.0,1.0,0.0,0.0,...,0.784983,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,e879756a-daff-4f72-9206-d96009861ac1,1.421779,1.473173,1.560174,0.0,0.0,0.0,1.0,0.0,0.0,...,0.671055,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,cbb0795d-716a-44f1-bc82-f8f8cb5db002,1.421779,1.204974,-0.752175,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.037875,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,e1e914fd-1e04-4148-9251-ed6a4dd6d3e7,1.421779,1.473173,1.560174,0.0,1.0,0.0,1.0,0.0,0.0,...,1.126769,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1987025d-39be-4120-a09a-8d89c0e8d151,1.421779,1.473173,-1.067496,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.379660,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
train_set.to_csv(root_folder+"Hanxi Liu/train_set.csv", header=True, index=None)
eval_set.to_csv(root_folder+"Hanxi Liu/eval_set.csv", header=True, index=None)

# Model

In [10]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split

In [19]:
def select_data(data_frame):
    '''Selects certain features from raw GCR records.'''
    
#     df_loc = fields.append('upgrade')
    df_selection = data_frame.iloc[:, 1:]
    df_selection.drop(columns=['redemption_day', 'customer_redemption_day', 'suspension_end_day', 'suspension_day', 'suspension_month'])
#     df_selection = data_frame.loc[:,["available_online_N","available_online_Y","battery_removable_N","battery_removable_Y","data_capable_1.0","data_capable_N","unlock_elegible_N","unlock_elegible_Y","bluetooth_N","bluetooth_Y","extd_warranty_1.0","extd_warranty_N","upgrade"]]
    return df_selection

def select_eval_data(data_frame):
    '''Selects certain features from raw GCR records.'''
    df_selection = data_frame
    df_selection.drop(columns=['redemption_day', 'customer_redemption_day', 'suspension_end_day','suspension_day','suspension_month'])
#     df_selection.drop(columns=['redemption_day', 'customer_redemption_day', 'suspension_end_day'])
#     df_selection = df_selection.drop(columns=['date_observed', 'suspension_start_date', 'suspension_end_date'])
#     df_selection = data_frame.loc[:,["line_id", "available_online_N","available_online_Y","battery_removable_N","battery_removable_Y","data_capable_1.0","data_capable_N","unlock_elegible_N","unlock_elegible_Y","bluetooth_N","bluetooth_Y","extd_warranty_1.0","extd_warranty_N"]]
    return df_selection

def get_usage_summary(df):
    '''Returns aggregate data usage for a line_id.'''
    data_kb= df.loc[:,['line_id', "total_kb"]]
    data_kb = data_kb.groupby('line_id')['total_kb'].sum().reset_index()
    data_kb.columns=['line_id','sum_total_kb']
    return data_kb

def preprocess_customer_info(df):
    '''Imputes null values in carrier and plan_name column with most frequent value.'''
    customer_info=df.loc[:,['line_id','carrier','plan_name']].drop_duplicates()
    customer_info['carrier'].fillna(customer_info['carrier'].mode()[0], inplace=True)
    customer_info['plan_name'].fillna(customer_info['plan_name'].mode()[0], inplace=True)
    customer_info=pd.get_dummies(customer_info,columns=['carrier','plan_name'],drop_first=True)
    return customer_info

def train_model(train_X,train_Y):
    '''Given a preprocessed training dataset, trains a simple logistic regression model and 
    returns the trained model object'''
#     lr_recipe = LogisticRegression(fit_intercept=True,class_weight='balanced')
#     lr_model = lr_recipe.fit(train_X,train_Y)
#     lr_recipe = RandomForestClassifier(max_depth=8, min_samples_leaf=20, min_samples_split=10,
#                         n_estimators=11, random_state=19)
#     lr_recipe = RandomForestClassifier(random_state=0,class_weight='balanced')
    rfc = RandomForestClassifier(max_depth=8, min_samples_leaf=20, min_samples_split=10,
                            n_estimators=11, random_state=19)
#     for n in range(2,10):
#         scores = cross_val_score(rfc, train_X, train_Y, cv=n)
#         print(n, scores, np.mean(scores))
        
#     ada = AdaBoostClassifier(random_state=19)
    lr_model = rfc.fit(train_X,train_Y)
#     lr_model = ada.fit(train_X,train_Y)
    return lr_model

def evaluate_model(model, test_X,test_Y):
    '''Given a model and preprocessed test dataset, returns the f1 value'''
    y_pred=model.predict(test_X)
#     f1 = f1_score(test_Y, y_pred, average='macro')
    f1 = f1_score(test_Y, y_pred)
    print('recall: ', recall_score(test_Y, y_pred))
    print('accuracy: ', accuracy_score(test_Y, y_pred))
    print('precision: ', precision_score(test_Y, y_pred))
    return round(f1,3)

def main(data_path, random_seed, test_ratio=0.3):
    '''The end to end model pipeline'''
    df =pd.read_csv(data_path,low_memory=True)    
    df_selected = select_data(df)
#     usage_summary=get_usage_summary(df_selected)
#     customer_info=preprocess_customer_info(df_selected)
#     line_ids=df_selected.loc[:,['line_id','upgrade']].drop_duplicates().reset_index(drop=True)
#     df_preprocessed=pd.merge(line_ids,usage_summary,on='line_id',how='inner')
#     df_preprocessed=pd.merge(df_preprocessed,customer_info,on='line_id')
#     df_Y=df_preprocessed['upgrade'].replace({'yes':1,'no':0})
#     df_X=df_preprocessed.drop(columns=['line_id','upgrade'])
#     print(df_selected.head(5))
    df_Y = df_selected.loc[:, ['upgrade']]
    df_X = df_selected.drop(columns=['upgrade'])
#     print(df_Y.head(5))
#     df_X  = df_selected.iloc[:, 0:-1]
#     df_Y = df_selected.iloc[:, -1]
    X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, test_size=test_ratio, random_state=random_seed)
    
    num_train = len(X_train)
    num_test = len(X_test)
    print(f"Train has {num_train}")
    print(f"Test has {num_test}")
    model = train_model(X_train,Y_train)
    f1 = evaluate_model(model, X_test, Y_test)
    return model,f1

def make_predictions(model,eval_data_path,submission_path):
    '''Given a model, eval data path and submission path, makes predictions and 
    saves the submissions to submission path.'''
#     eval_data = pd.read_csv(eval_data_path,low_memory=True) 
#     eval_data = pd.read_csv(eval_data_path) 
    eval_data = eval_set
    df_selected = select_eval_data(eval_data)
#     usage_summary=get_usage_summary(df_selected)
#     customer_info=preprocess_customer_info(df_selected)
    line_ids=df_selected.loc[:,['line_id']].drop_duplicates().reset_index(drop=True)
#     df_preprocessed=pd.merge(line_ids,usage_summary,on='line_id',how='inner')
#     df_preprocessed=pd.merge(df_preprocessed,customer_info,on='line_id')
#     eval_X=df_preprocessed.drop(columns=['line_id'])
    eval_X=df_selected.drop(columns=['line_id'])
    predictions=pd.DataFrame(line_ids,columns=['line_id'])
    predictions['upgrade']=model.predict(eval_X)
    #predictions.to_csv(submission_path,header=True,index=None)
    #print(f"submission saved to {submission_path}")

In [20]:
# This path will be active after the launch of the hackathon  / day-day-up-unsw / s3://tf-trachack-notebooks/day-day-up-unsw/jupyter/jovyan/
teamname = 'day-day-up-unsw' # 'trachack-a-groups-admin-py-tracfone'
data_folder= 's3://tf-trachack-data/212/'
# change root_folder to your team's root folder
# s3://tf-trachack-notebooks/<this should be replaced by team name as provided in EMAIL>/jupyter/jovyan/
root_folder='s3://tf-trachack-notebooks/'+teamname+'/jupyter/jovyan/'
#data_path = root_folder+'Hanxi Liu/dev-sample.csv'
data_path = root_folder+"Hanxi Liu/train_set.csv"
seed = 125
model,f1 = main(data_path, seed)
print(f"f1-score: {f1}")

Train has 39107
Test has 16761
recall:  0.7865612648221344
accuracy:  0.8871785692977746
precision:  0.7958231504110198
f1-score: 0.791
